<figure>
  <img src="https://raw.githubusercontent.com/shadowkshs/DimABSA2026/refs/heads/main/banner.png" width="100%">
</figure>

# SemEval-2026 Task 2 & 3 (Track A: DimABSA)
# Subtask 2: DimASTE - Dimensional Aspect Sentiment Triplet Extraction
# Subtask 3: DimASQE - Dimensional Aspect Sentiment Quadruplets Extraction

-----

## Starter Notebook
LLM-based finetune for Dimensional Aspect Sentiment Triplet (and Quadruplet) Extraction

## Introduction:

You are welcome to participate in our SemEval Shared Task!

In this starter notebook, we guide you through the process of fine-tuning a pre-trained language model on sample training data to build a dimensional sentiment extraction model.  
This notebook is adapted from a HuggingFace-style implementation for similar tasks.

### Outline:
- Installation and importation of necessary libraries Setting up the project parameters. Running training and evaluation Before you start:

- It is strongly advised that you use a GPU to speed up training. To do this, go to the "Runtime" menu in Colab, select "Change runtime type" and then in the popup menu, choose "GPU" in the "Hardware accelerator" box.

### NB:
- This notebook aims to help you become familiar with fine-tuning language models for dimensional sentiment tasks.  
- You are encouraged to extend or modify it to obtain competitive performance.
- This notebook will take about 2 to 2.5 hours to run. It may shut down if your Colab GPU quota is insufficient.

### Languages and Domains:
#### Track A: Subtask 2 & 3
- eng_restaurant
- eng_laptop
- jpn_hotel
- rus_restaurant
- tat_restaurant
- ukr_restaurant
- zho_restaurant
- zho_laptop

### Model:
This Starter Notebook uses the Qwen3-4B-Instruct pretrained language model, adapted through Unsloth for efficient fine-tuning.

The model is a multilingual decoder-based LLM with strong instruction-following ability, and it is provided in bnb 4-bit quantized form, allowing training on limited GPU resources such as Google Colab.

You can find the model here:
https://huggingface.co/unsloth/Qwen3-4B-Instruct-2507-bnb-4bit

If you need alternative models (e.g., larger Qwen variants, or other multilingual LLMs), you may explore them on Hugging Face:
https://huggingface.co/unsloth/models

### Step 1: Install unsloth package

In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

### Step 2: Load the competition data

In [ ]:
import os
import re
import json
from datasets import load_dataset

#task config
subtask = "subtask_2"# subtask_2 or subtask_3
task = "task2" # task2 or task3
lang = "eng" #chang the language you want to test
domain = "restaurant" #change what domain you want to test

train_url = f"https://raw.githubusercontent.com/DimABSA/DimABSA2026/refs/heads/main/task-dataset/track_a/{subtask}/{lang}/{lang}_{domain}_train_alltasks.jsonl"
predict_url = f"https://raw.githubusercontent.com/DimABSA/DimABSA2026/refs/heads/main/task-dataset/track_a/{subtask}/{lang}/{lang}_{domain}_dev_{task}.jsonl"

#load train data from url
dataset = load_dataset("json", data_files=train_url)

Generating train split: 0 examples [00:00, ? examples/s]

### Display the data info

In [ ]:
print(dataset)
print(dataset['train'][0])

DatasetDict({
    train: Dataset({
        features: ['ID', 'Text', 'Quadruplet'],
        num_rows: 2284
    })
})
{'ID': 'rest16_quad_dev_1', 'Text': "ca n ' t wait wait for my next visit .", 'Quadruplet': [{'Aspect': 'NULL', 'Opinion': 'NULL', 'Category': 'RESTAURANT#GENERAL', 'VA': '6.75#6.38'}]}


### Step 3: Design prompt template

In [ ]:
# task 2 prompt template covert
if task == "task2":
  instruction = '''Below is an instruction describing a task, paired with an input that provides additional context. Your goal is to generate an output that correctly completes the task.

### Instruction:
Given a textual instance [Text], extract all (A, O, VA) triplets, where:
- A is an Aspect term (a phrase describing an entity mentioned in [Text])
- O is an Opinion term
- VA is a Valence–Arousal score in the format (valence#arousal)

Valence ranges from 1 (negative) to 9 (positive),
Arousal ranges from 1 (calm) to 9 (excited).

### Example:
Input:
[Text] average to good thai food, but terrible delivery.

Output:
[Triplet] (thai food, average to good, 6.75#6.38), (delivery, terrible, 2.88#6.62)

### Question:
Now complete the following example:
Input:
'''

  def convert(x):
      text = x["Text"]
      quads = x.get("Quadruplet", [])
      answer = ", ".join([
          f"({q['Aspect']}, {q['Opinion']}, {q['VA']})"
          for q in quads
      ])
      prompt = instruction + "[Text] " + text + "\n\nOutput:"
      return {"text": f"<|user|>\n{prompt}\n<|assistant|>\n{answer}"}

# task 3 prompt template covert, with task3 predefine entity and attribute labels.
elif task == "task3":
  rest_entity = 'RESTAURANT, FOOD, DRINKS, AMBIENCE, SERVICE, LOCATION'
  rest_attribute = 'GENERAL, PRICES, QUALITY, STYLE_OPTIONS, MISCELLANEOUS'

  laptop_entity = 'LAPTOP, DISPLAY, KEYBOARD, MOUSE, MOTHERBOARD, CPU, FANS_COOLING, PORTS, MEMORY, POWER_SUPPLY, OPTICAL_DRIVES, BATTERY, GRAPHICS, HARD_DISK, MULTIMEDIA_DEVICES, HARDWARE, SOFTWARE, OS, WARRANTY, SHIPPING, SUPPORT, COMPANY'
  laptop_attribute = 'GENERAL, PRICE, QUALITY, DESIGN_FEATURES, OPERATION_PERFORMANCE, USABILITY, PORTABILITY, CONNECTIVITY, MISCELLANEOUS'

  hotel_entity = 'HOTEL, ROOMS, FACILITIES, ROOM_AMENITIES, SERVICE, LOCATION, FOOD_DRINKS'
  hotel_attribute = 'GENERAL, PRICE, COMFORT, CLEANLINESS, QUALITY, DESIGN_FEATURES, STYLE_OPTIONS, MISCELLANEOUS'

  finance_entity = 'MARKET, COMPANY, BUSINESS, PRODUCT'
  finance_attribute = 'GENERAL, SALES, PROFIT, AMOUNT, PRICE, COST'

  entity_attribute_map = {
      'restaurant': (rest_entity, rest_attribute),
      'laptop': (laptop_entity, laptop_attribute),
      'hotel': (hotel_entity, hotel_attribute),
      'finance': (finance_entity, finance_attribute),
  }

  entity_label, attribute_label = entity_attribute_map[domain]

  instruction = f'''Below is an instruction describing a task, paired with an input that provides additional context. Your goal is to generate an output that correctly completes the task.

### Instruction:
Given a textual instance [Text], extract all (A, C, O, VA) quadruplets, where:
- A is an Aspect term (a phrase describing an entity mentioned in [Text])
- C is a Category label (e.g. FOOD#QUALITY)
- O is an Opinion term
- VA is a Valence–Arousal score in the format (valence#arousal)

Valence ranges from 1 (negative) to 9 (positive),
Arousal ranges from 1 (calm) to 9 (excited).

### Label constraints:
[Entity Labels] ({entity_label})
[Attribute Labels] ({attribute_label})

### Example:
Input:
[Text] average to good thai food, but terrible delivery.

Output:
[Quadruplet] (thai food, FOOD#QUALITY, average to good, 6.75#6.38),
             (delivery, SERVICE#GENERAL, terrible, 2.88#6.62)

### Question:
Now complete the following example:
Input:
'''

  def convert(x):
      text = x["Text"]
      quads = x.get("Quadruplet", [])
      answer = ", ".join([
          f"({q['Aspect']}, {q['Category']}, {q['Opinion']}, {q['VA']})"
          for q in quads
      ])
      prompt = instruction + "[Text] " + text + "\n\nOutput:"
      return {"text": f"<|user|>\n{prompt}\n<|assistant|>\n{answer}"}


# covert dataset to train template
train_dataset = dataset["train"].map(convert)


Map:   0%|          | 0/2284 [00:00<?, ? examples/s]

In [ ]:
#show your template text
print(train_dataset["text"][0])

<|user|>
Below is an instruction describing a task, paired with an input that provides additional context. Your goal is to generate an output that correctly completes the task.

### Instruction:
Given a textual instance [Text], extract all (A, O, VA) triplets, where:
- A is an Aspect term (a phrase describing an entity mentioned in [Text])
- O is an Opinion term
- VA is a Valence–Arousal score in the format (valence#arousal)

Valence ranges from 1 (negative) to 9 (positive),
Arousal ranges from 1 (calm) to 9 (excited).

### Example:
Input:
[Text] average to good thai food, but terrible delivery.
[Aspect] thai food, delivery

Output:
[Triplet] (thai food, average to good, 6.75#6.38), (delivery, terrible, 2.88#6.62)

### Question:
Now complete the following example:
Input:
[Text] ca n ' t wait wait for my next visit .

Output:
<|assistant|>
(NULL, NULL, 6.75#6.38)


### Step 4: Load the LLM from Hugging Face and apply LoRA for fine-tuning


In [ ]:
from unsloth import FastLanguageModel

model_id = "unsloth/Qwen3-4B-Instruct-2507-bnb-4bit" # you can change the model here

# tokenizer and model setting
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = 512, # DimASBA Task usually less then 512 tokens.
    load_in_4bit = True,
)

# lora setting
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 16,
    lora_dropout = 0.05,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.3: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.65G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.11.3 patched 36 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


### Step 5: Start LoRA Fine-Tuning


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = 512,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 20,
        num_train_epochs = 2, # epoches
        learning_rate = 1e-4, #　learning rate
        logging_steps = 50,
        save_steps = 100,
        fp16 = True,
        bf16 = False,
        report_to = "none",
        output_dir = "./Lora", # save Lora
    ),
)

trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/2284 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,284 | Num Epochs = 2 | Total steps = 1,142
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 11,796,480 of 4,034,264,576 (0.29% trained)


Step,Training Loss
50,1.533600


Unsloth: Input IDs of shape torch.Size([1, 769]) with length 769 > the model's max sequence length of 512.
We shall truncate it ourselves. It's imperative if you correct this issue first.


Unsloth: Will smartly offload gradients to save VRAM!


TorchRuntimeError: Dynamo failed to run FX node with fake tensors: call_function <function cross_entropy at 0x797d451e4860>(*(), **{'input': GradTrackingTensor(lvl=1, value=
    FakeTensor(..., device='cuda:0', size=(s97, 151936))
), 'target': GradTrackingTensor(lvl=1, value=
    FakeTensor(..., device='cuda:0', size=(s7,), dtype=torch.int64)
), 'reduction': 'sum'}): got ValueError('Expected input batch_size (s97) to match target batch_size (s7).')

from user code:
   File "/usr/local/lib/python3.12/dist-packages/unsloth_zoo/fused_losses/cross_entropy_loss.py", line 276, in accumulate_chunk
    (chunk_loss, (unscaled_loss,)) = torch.func.grad_and_value(
  File "/usr/local/lib/python3.12/dist-packages/torch/_functorch/apis.py", line 441, in wrapper
    return eager_transforms.grad_and_value_impl(
  File "/usr/local/lib/python3.12/dist-packages/torch/_functorch/vmap.py", line 48, in fn
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.12/dist-packages/torch/_functorch/eager_tr

### Step 6: Run Inference and Extract Structured Sentiment Outputs

In [ ]:
# load dev json to predict
predict_dataset = load_dataset("json", data_files=predict_url)

# convert text to prompt
def format_dataset(x):
    text = x["Text"]
    final_prompt = instruction + '[Text] ' + text + '\n\nOutput:'
    return [
        {"role": "user", "content": final_prompt}
    ]

# extract answer
def extract_answer(text,task):
  result = []
  if task == "task2":
    pattern = r'\(([^,]+),\s*([^,]+),\s*([\d.]+#[\d.]+)\)'
    matches = re.findall(pattern, text)

    for aspect, opinion, va in matches:
        meta_triplet = {}
        meta_triplet["Aspect"] = aspect.strip()
        meta_triplet["Opinion"] = opinion.strip()
        meta_triplet["VA"] = va
        result.append(meta_triplet)

  elif task == "task3":
    pattern = r'\(([^,]+),\s*([^,]+),\s*([^,]+),\s*([^)]+)\)'
    matches = re.findall(pattern, text)

    for aspect, category, opinion, va in matches:
        meta_quadra = {}
        meta_quadra["Aspect"] = aspect.strip()
        meta_quadra["Category"] = category.strip()
        meta_quadra["Opinion"] = opinion.strip()
        meta_quadra["VA"] = va
        result.append(meta_quadra)
  else:
    raise ValueError("Invalid task")

  return result

# Perform inference
results = []
for i, sample in enumerate(predict_dataset["train"]):
    messages = format_dataset(sample)

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False,
    )

    result = model.generate(
        **tokenizer(text, return_tensors="pt").to("cuda"),
        max_new_tokens=1024,
        temperature=0.7, top_p=0.8, top_k=20,
    )

    decoded = tokenizer.decode(result[0])
    extracted_text = decoded.split("\n")[-1]

    key = "Triplet" if task == "task2" else "Quadruplet"

    dump_data = {
        "ID": sample.get("ID", f"sample_{i}"),
        "Text": sample["Text"],
        key: extract_answer(extracted_text, task),
    }

    print(dump_data)
    results.append(dump_data)


### Step 7: Save prediction results

In [ ]:
import json
import os
import zipfile
from google.colab import files

# resolve output name
out_name = f"pred_{lang}_{domain}.jsonl"

# ensure output folder
os.makedirs(subtask, exist_ok=True)

# JSONL file path
jsonl_path = os.path.join(subtask, out_name)

# write JSONL
with open(jsonl_path, "w", encoding="utf-8") as f:
    for item in results:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

# zip name is subtask folder name
zip_name = f"{subtask}.zip"

# zip the folder
with zipfile.ZipFile(zip_name, "w", zipfile.ZIP_DEFLATED) as zf:
    for root, _, files_in_dir in os.walk(subtask):
        for file in files_in_dir:
            full_path = os.path.join(root, file)
            zf_path = os.path.relpath(full_path, ".")
            zf.write(full_path, zf_path)

# download
files.download(zip_name)